# Regression Modeling

## Load data

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Load and prepare the dataset
df = pd.read_feather('../data/spotify_2000_2020.feather')
df['is_hit'] = (df['popularity'] > 80).astype(int)


## Step 1: Linear Regression to Predict Popularity

In [2]:
# Simple linear regression using statsmodels
model = smf.ols('popularity ~ danceability', data=df).fit()

# Print summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             popularity   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     884.5
Date:                Thu, 08 May 2025   Prob (F-statistic):          2.43e-192
Time:                        18:20:38   Log-Likelihood:            -1.5810e+05
No. Observations:               41656   AIC:                         3.162e+05
Df Residuals:                   41654   BIC:                         3.162e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       49.4894      0.187    264.090   

### Key Results:

- Intercept (β₀): 49.49 → When danceability = 0, predicted popularity ≈ 49.5

- Slope (β₁): 9.05 → For every 1-unit increase in danceability, popularity increases by ~9.05 points on average

- p-value for danceability: < 0.0001 → The relationship is statistically significant

- R-squared = 0.021 → Danceability explains only 2.1% of the variation in popularity

It’s statistically significant
But not practically strong, because R² is just 0.021 → meaning danceability alone explains only 2.1% of popularity variation.

So: it’s real, but not enough by itself.

## Step 2: Multiple Linear Regression

In [3]:
# Multiple regression formula
features = ['danceability', 'energy', 'valence', 'loudness', 'tempo']
formula = 'popularity ~ ' + ' + '.join(features)

# Fit the model
multi_model = smf.ols(formula=formula, data=df).fit()

# Show summary
multi_model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             popularity   R-squared:                       0.049
Model:                            OLS   Adj. R-squared:                  0.049
Method:                 Least Squares   F-statistic:                     427.8
Date:                Thu, 08 May 2025   Prob (F-statistic):               0.00
Time:                        18:34:13   Log-Likelihood:            -1.5749e+05
No. Observations:               41656   AIC:                         3.150e+05
Df Residuals:                   41650   BIC:                         3.151e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       53.4990      0.495    108.169      0.000      52.530      54.468
danceability    13.5715      0.368     36.919      0.000      12.851      14.292
energy          -3.9210      0.362    -10.832      0.000      -4.631      -3.212
valence         -7.4858      0.256    -29.282      0.000      -7.987      -6.985
loudness         0.1894      0.018     10.371      0.000       0.154       0.225
tempo            0.0079      0.002      4.517      0.000       0.004       0.011
==============================================================================
Omnibus:                     1811.905   Durbin-Watson:                   0.522
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5556.845
Skew:                          -0.143   Prob(JB):                         0.00
Kurtosis:                       4.766   Cond. No.                     1.49e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.49e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Step 3: Logistic Regression: Predicting Whether a Song is a Hit

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Features and target
X = df[['danceability', 'energy', 'valence', 'loudness', 'tempo']]
y = df['is_hit']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
# Fit the logistic regression model
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)

# Predict on test set
y_pred = log_model.predict(X_test)

# Evaluate performance
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      8242
           1       0.00      0.00      0.00        90

    accuracy                           0.99      8332
   macro avg       0.49      0.50      0.50      8332
weighted avg       0.98      0.99      0.98      8332

Confusion Matrix:
 [[8242    0]
 [  90    0]]


/home/agnes/miniconda3/envs/data-science-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/agnes/miniconda3/envs/data-science-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/agnes/miniconda3/envs/data-science-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

- The model **predicted all songs as not hits**, completely missing the rare hit class.
- **Accuracy = 99%**, but this is misleading due to class imbalance.
- Precision, recall, and F1-score for hit class are all **0.00**, meaning the model is not useful in its current form.

Next step: Apply `class_weight='balanced'` to handle imbalance or use resampling techniques like SMOTE in model tuning.

In [7]:
# Re-train with class_weight='balanced'
log_model_balanced = LogisticRegression(class_weight='balanced', max_iter=1000)
log_model_balanced.fit(X_train, y_train)

# Predict on test set
y_pred_balanced = log_model_balanced.predict(X_test)

# Evaluate performance
print("Classification Report (Balanced):\n", classification_report(y_test, y_pred_balanced, zero_division=0))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_balanced))


Classification Report (Balanced):
               precision    recall  f1-score   support

           0       0.99      0.67      0.80      8242
           1       0.02      0.67      0.04        90

    accuracy                           0.67      8332
   macro avg       0.51      0.67      0.42      8332
weighted avg       0.98      0.67      0.79      8332

Confusion Matrix:
 [[5526 2716]
 [  30   60]]


- Using `class_weight='balanced'`, the model now correctly identified **67% of hits** (recall = 0.67), compared to 0% before.
- It correctly predicted **60 out of 90 hits**, a substantial improvement.
- Precision for hits is low (0.02), meaning it predicted many false positives — but this is expected when the model is optimized for recall on rare classes.
- Accuracy dropped to 67%, but this tradeoff is acceptable when our goal is **not to miss hits**.

### Conclusion:
This model provides a meaningful baseline for predicting hit songs in an imbalanced setting. For better performance, next steps could include:
- Threshold tuning
- ROC-AUC analysis
- More advanced models (e.g., Random Forest, XGBoost)
- Resampling methods like SMOTE